# RNN 실습 - 텐서플로우에서 LSTM 및 GRU 사용

1. RNN 모형 및 구성방법
1. 텐서플로우에서 지원하는 RNN 'Cell' 유형
1. 유형별 특성 테스트 위한 코드 구성
1. Test #1 - Vanila RNN
1. Test #2 - Basic LSTM
1. Test #3 - GRU
1. Test #4 - LSTMCell + forget_bias
1. Test #5 - LayerNormBasicLSTMCell
1. Test #6 - LayerNormBasicLSTMCell - what's wrong?
1. 정리

In [1]:
!rm -fr logdir
!mkdir -p logdir

In [2]:
%load_ext do_not_print_href
%matplotlib inline
from __future__ import print_function, division
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## RNN 모형 (복습)


<img  src="rnn.jpg" style="width:55.5rem"/>

<center>이미지 출처: http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/</center>

\begin{align}
s_t & = tanh(Ux_t + Ws_{t-1}) \\
o_t & = softmax(Vs_t) \\
\end{align}


### 하지만 텐서플로우 RNN 은...

<img  src="Selection_20170914_161757_e791.png"/>


- **_V_** 에 해당하는 구조가 없음

- **_softmax()_** 도 없음

- **_V_** 와 **_softmax()_** 는 필요할 때만 만들어서 붙이면 됨 ( `tf.layers.dense`, `tf.nn.softmax` )



## RNN 모델 구성 (복습)

- [`tf.contrib.rnn.BasicRNNCell`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/basicrnncell)

<code>
  `__init__`(
    <span style="color:red">num_units,</span>
    activation=None,
    reuse=None
  )
</code>


- [`tf.nn.dynamic_rnn`](http://devdocs.io/tensorflow~python/tf/nn/dynamic_rnn)

<code>
  dynamic_rnn(
    <span style="color:red">cell,</span>
    <span style="color:red">inputs,</span>
    <span style="color:red">sequence_length=None,</span>
    initial_state=None,
    dtype=None,
    parallel_iterations=None,
    swap_memory=False,
    time_major=False,
    scope=None
  )
</code>


- RNN 구성


<code>
    cell            = tf.contrib.rnn.<b style="color:red">BasicRNNCell</b>(
                        num_hidden_units)
    last, states    = tf.nn.dynamic_rnn(
                        cell, 
                        inputs, 
                        <i style="color:red">sequence_length=sequence_length,</i>
                        dtype=tf.float32)
</code>



## RNN Cell 종류

- [`tf.contrib.rnn.BasicRNNCell`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/basicrnncell)

<code>
    ...
</code>

- [`tf.contrib.rnn.BasicLSTMCell`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/basiclstmcell)

<code>
    `__init__`(
        <span style="color:red">num_units,</span>
        forget_bias=1.0,
        state_is_tuple=True,
        activation=None,
        reuse=None
    )
</code>

- [`tf.contrib.rnn.LSTMCell`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/lstmcell)

<code>
    `__init__`(
        <span style="color:red">num_units,</span>
        use_peepholes=False,
        cell_clip=None,
        initializer=None,
        num_proj=None,
        proj_clip=None,
        num_unit_shards=None,
        num_proj_shards=None,
        forget_bias=1.0,
        state_is_tuple=True,
        activation=None,
        reuse=None
    )
</code>

- [`tf.contrib.rnn.GRUCell`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/grucell)

<code>
    `__init__`(
        <span style="color:red">num_units,</span>
        activation=None,
        reuse=None,
        kernel_initializer=None,
        bias_initializer=None
    )
</code>

- [`tf.contrib.rnn.LayerNormBasicLSTMCell`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/layernormbasiclstmcell)

> LSTM unit with **layer normalization** and **recurrent dropout.**

<code>
    `__init__`(
        <span style="color:red">num_units,</span>
        forget_bias=1.0,
        input_size=None,
        activation=tf.tanh,
        layer_norm=True,
        norm_gain=1.0,
        norm_shift=0.0,
        dropout_keep_prob=1.0,
        dropout_prob_seed=None,
        reuse=None
    )
</code>

- 기타 여러가지 Wrapper 지원 - https://www.tensorflow.org/api_guides/python/contrib.rnn

In [3]:
from tensorflow.examples.tutorials.mnist.input_data \
    import read_data_sets

mnist = read_data_sets('./mnist', one_hot=False)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [4]:
INPUT_UNITS = 28
NUM_HIDDEN_UNITS = 31

BATCH_SIZE = 128
MAX_SEQ_LEN = 28

In [5]:
class MnistRnn:
    def __init__(self, 
                 inputs, 
                 labels, 
                 input_units, 
                 num_hidden_units, 
                 batch_size, 
                 max_seq_len,
                 rnn_cell_class = tf.contrib.rnn.BasicRNNCell,
                 # 여기 이후의 인수는 잠깐 무시 해 주세요
                 add_check = False,
                 lr = 0.001,
                 use_grad_clip = False):
        '''
        inputs: in shape [batch_size, max_seq_len, input_size]
        labels: in shape [batch_size]
        '''

        cell            = rnn_cell_class(num_hidden_units)
        sequence_length = [max_seq_len] * batch_size
        last, states    = tf.nn.dynamic_rnn(
                            cell, 
                            inputs, 
                            sequence_length=sequence_length, 
                            dtype=tf.float32)

        # max_seq_len 축으로 0~27 까지 값 중에 
        # 0~26 때의 출력 값은 사용하지 않음
        rnn_output = last[:,max_seq_len-1,:]
        # outputs shape will be: [batch_size, 10]
        outputs    = tf.layers.dense(rnn_output, 10)
        loss       = tf.losses.sparse_softmax_cross_entropy(
                        labels, outputs)
        
        if use_grad_clip:
            tvars_     = tf.trainable_variables()
            grads_, _  = tf.clip_by_global_norm(
                            tf.gradients(
                                loss,
                                tvars_),
                            5.0)
            optimize   = \
                tf.train.AdamOptimizer(learning_rate=lr). \
                            apply_gradients(zip(grads_, tvars_))
        else:
            optimize   = \
                tf.train.AdamOptimizer(learning_rate=lr). \
                            minimize(loss)

        # accuracy
        preds    = tf.argmax(outputs, axis=1)
        errors   = tf.count_nonzero(labels - preds)
        accuracy = 1.0 - tf.cast(errors,tf.float32) / \
                         tf.cast(tf.size(preds),tf.float32)

        # 클래스 객체 외부에서 참고할 수 있도록 속성으로 저장
        self.outputs        = outputs
        self.loss           = loss
        self.optimize       = optimize
        self.accuracy       = accuracy
        
        # check_numerics
        self.check = [tf.check_numerics(
                        t,
                        'check_numerics: {}'.format(t.name)) \
                      for t in tf.gradients(
                                  loss,
                                  tf.trainable_variables()) \
                      if t is not None] \
                     if add_check \
                     else tf.constant(1.0)

In [6]:
train_loop_count = mnist.train.num_examples // BATCH_SIZE
test_loop_count  = mnist.test.num_examples // BATCH_SIZE

train_loop_count, test_loop_count

(429, 78)

In [7]:
def train(inputs, labels, max_epochs, train_writer, test_writer):
    step = 0
    for ep in range(max_epochs):

        train_elapsed = []
        train_losses = []
        train_accuracy = []
        for i in range(train_loop_count):
            t_start     = time.time()
            offs        = i * BATCH_SIZE
            batch_input = \
                    mnist.train.images[offs:offs+BATCH_SIZE,:]
            batch_input = \
                    batch_input.reshape(
                            [BATCH_SIZE,
                               MAX_SEQ_LEN,
                               INPUT_UNITS])
            batch_label = \
                    mnist.train.labels[offs:offs+BATCH_SIZE]
            optimize, loss, accuracy, _ = \
                sess.run([model.optimize,
                          model.loss,
                          model.accuracy,
                          model.check],
                         feed_dict = {
                          inputs: batch_input,
                          labels: batch_label })
            train_losses.append(loss)
            train_accuracy.append(accuracy)
            t_elapsed   = time.time() - t_start
            train_elapsed.append(t_elapsed)

            step += 1
            summary = tf.Summary(
                value=[
                    tf.Summary.Value(
                        tag='train_accuracy',
                        simple_value=accuracy
                    ),
                    tf.Summary.Value(
                        tag='loss',
                        simple_value=loss
                    ),
                ]
            )
            train_writer.add_summary(summary,global_step=step)

            if step % 250 == 0:
                print(('[trn] ep {:d}, step {:d}, ' + 
                       'loss {:f}, accu {:f}, ' + 
                       'sec/iter {:f}').format(
                    ep + 1,
                    step,
                    np.mean(train_losses),
                    np.amin(train_accuracy),
                    np.mean(train_elapsed)))
                train_losses = []
                train_accuracy = []
                train_elapsed = []
                
        train_writer.flush()

        test_elapsed  = []
        test_accuracy = []
        
        for i in range(test_loop_count):
            t_start     = time.time()
            offs        = i * BATCH_SIZE
            batch_input = mnist.test.images[offs:offs+BATCH_SIZE,:]
            batch_input = batch_input.reshape(
                            [BATCH_SIZE,
                               MAX_SEQ_LEN,
                               INPUT_UNITS])
            batch_label = mnist.test.labels[offs:offs+BATCH_SIZE]
            accuracy, = \
                sess.run([model.accuracy],
                         feed_dict = {
                          inputs: batch_input,
                          labels: batch_label })
            test_accuracy.append(accuracy)
            t_elapsed   = time.time() - t_start
            test_elapsed.append(t_elapsed)

            step += 1
            
        if len(test_accuracy) > 0:
            print(('[tst] ep {:d}, step {:d}, ' +
                   'accu {:f}, sec/iter {:f}').format(
                ep + 1,
                step,
                np.amin(test_accuracy),
                np.mean(test_elapsed)))

            summary = tf.Summary(
                value=[
                    tf.Summary.Value(
                        tag='test_accuracy',
                        simple_value=np.amin(test_accuracy)
                    ),
                ]
            )
            test_writer.add_summary(summary,global_step=step)
            test_writer.flush()


## Test #1 - Vanila RNN

<div  style="border:1px solid black;border-radius:5px">
<code>
    cell = tf.contrib.rnn.BasicRNNCell(num_hidden_units)

</code>
</div>

In [8]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')


model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 tf.contrib.rnn.BasicRNNCell)

In [9]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter('logdir/train_basic_rnn',
                                     graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter('logdir/test_basic_rnn',
                                     graph=tf.get_default_graph())

train(inputs_, labels_, 10, train_writer, test_writer)

[trn] ep 1, step 250, loss 1.550250, accu 0.085938, sec/iter 0.007951
[tst] ep 1, step 507, accu 0.468750, sec/iter 0.002797
[trn] ep 2, step 750, loss 0.987132, accu 0.562500, sec/iter 0.007306
[tst] ep 2, step 1014, accu 0.539062, sec/iter 0.002285
[trn] ep 3, step 1250, loss 0.765099, accu 0.625000, sec/iter 0.007807
[tst] ep 3, step 1521, accu 0.648438, sec/iter 0.004670
[trn] ep 4, step 1750, loss 0.632159, accu 0.648438, sec/iter 0.007768
[tst] ep 4, step 2028, accu 0.656250, sec/iter 0.003359
[trn] ep 5, step 2250, loss 0.562881, accu 0.664062, sec/iter 0.009445
[tst] ep 5, step 2535, accu 0.671875, sec/iter 0.005474
[trn] ep 6, step 2750, loss 0.504307, accu 0.695312, sec/iter 0.008914
[tst] ep 6, step 3042, accu 0.679688, sec/iter 0.003352
[trn] ep 7, step 3250, loss 0.462129, accu 0.718750, sec/iter 0.009098
[tst] ep 7, step 3549, accu 0.664062, sec/iter 0.005065
[trn] ep 8, step 3750, loss 0.429448, accu 0.757812, sec/iter 0.009314
[tst] ep 8, step 4056, accu 0.710938, sec/i

In [10]:
# !tensorboard --ip 0.0.0.0 --logdir logdir

## Test #2 - Basic LSTM

<div  style="border:1px solid black;border-radius:5px">
<code>
    cell = tf.contrib.rnn.BasicLSTMCell(num_hidden_units)

</code>
</div>

In [11]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 tf.contrib.rnn.BasicLSTMCell)

In [12]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter('logdir/train_basic_lstm',
                                     graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter('logdir/test_basic_lstm',
                                     graph=tf.get_default_graph())

train(inputs_, labels_, 10, train_writer, test_writer)

[trn] ep 1, step 250, loss 1.499541, accu 0.039062, sec/iter 0.013030
[tst] ep 1, step 507, accu 0.695312, sec/iter 0.005082
[trn] ep 2, step 750, loss 0.469044, accu 0.671875, sec/iter 0.013055
[tst] ep 2, step 1014, accu 0.796875, sec/iter 0.004687
[trn] ep 3, step 1250, loss 0.304381, accu 0.773438, sec/iter 0.013056
[tst] ep 3, step 1521, accu 0.851562, sec/iter 0.006747
[trn] ep 4, step 1750, loss 0.228537, accu 0.835938, sec/iter 0.013932
[tst] ep 4, step 2028, accu 0.867188, sec/iter 0.005018
[trn] ep 5, step 2250, loss 0.196097, accu 0.843750, sec/iter 0.012640
[tst] ep 5, step 2535, accu 0.867188, sec/iter 0.004851
[trn] ep 6, step 2750, loss 0.166952, accu 0.882812, sec/iter 0.012917
[tst] ep 6, step 3042, accu 0.875000, sec/iter 0.004824
[trn] ep 7, step 3250, loss 0.147182, accu 0.898438, sec/iter 0.014815
[tst] ep 7, step 3549, accu 0.867188, sec/iter 0.005175
[trn] ep 8, step 3750, loss 0.133158, accu 0.906250, sec/iter 0.013107
[tst] ep 8, step 4056, accu 0.867188, sec/i

In [13]:
# !tensorboard --ip 0.0.0.0 --logdir logdir

## Test #3 - GRU

<div  style="border:1px solid black;border-radius:5px">
<code>
    cell = tf.contrib.rnn.GRUCell(num_hidden_units)

</code>
</div>

In [14]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 tf.contrib.rnn.GRUCell)

In [15]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter(
                'logdir/train_gru',
                graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter(
                'logdir/test_gru',
                graph=tf.get_default_graph())

train(inputs_, labels_, 10, train_writer, test_writer)

[trn] ep 1, step 250, loss 1.550183, accu 0.117188, sec/iter 0.014488
[tst] ep 1, step 507, accu 0.679688, sec/iter 0.005059
[trn] ep 2, step 750, loss 0.464381, accu 0.703125, sec/iter 0.014379
[tst] ep 2, step 1014, accu 0.820312, sec/iter 0.004900
[trn] ep 3, step 1250, loss 0.284551, accu 0.820312, sec/iter 0.014348
[tst] ep 3, step 1521, accu 0.859375, sec/iter 0.004766
[trn] ep 4, step 1750, loss 0.216922, accu 0.828125, sec/iter 0.015113
[tst] ep 4, step 2028, accu 0.882812, sec/iter 0.004711
[trn] ep 5, step 2250, loss 0.180984, accu 0.851562, sec/iter 0.014021
[tst] ep 5, step 2535, accu 0.890625, sec/iter 0.005294
[trn] ep 6, step 2750, loss 0.158896, accu 0.835938, sec/iter 0.015809
[tst] ep 6, step 3042, accu 0.882812, sec/iter 0.004615
[trn] ep 7, step 3250, loss 0.140687, accu 0.875000, sec/iter 0.015920
[tst] ep 7, step 3549, accu 0.890625, sec/iter 0.007297
[trn] ep 8, step 3750, loss 0.126121, accu 0.898438, sec/iter 0.018148
[tst] ep 8, step 4056, accu 0.906250, sec/i

In [16]:
# !tensorboard --ip 0.0.0.0 --logdir logdir

## Test #4 - LSTMCell + forget_bias

<div  style="border:1px solid black;border-radius:5px">
<code>
    cell = tf.contrib.rnn.BasicLSTMCell(
            num_hidden_units,
            forget_bias=lstm_forget_bias)
 
</code>
</div>

In [17]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

lstm_with_forget_bias = lambda num_hidden_units: \
    tf.contrib.rnn.BasicLSTMCell(
            num_hidden_units,
            forget_bias=5.0)

model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 lstm_with_forget_bias)

In [18]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter('logdir/train_lstm_forget_bias',
                                     graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter('logdir/test_lstm_forget_bias',
                                     graph=tf.get_default_graph())

In [19]:
train(inputs_, labels_, 10, train_writer, test_writer)

[trn] ep 1, step 250, loss 1.470503, accu 0.054688, sec/iter 0.013168
[tst] ep 1, step 507, accu 0.617188, sec/iter 0.005363
[trn] ep 2, step 750, loss 0.543548, accu 0.664062, sec/iter 0.013188
[tst] ep 2, step 1014, accu 0.703125, sec/iter 0.007593
[trn] ep 3, step 1250, loss 0.369156, accu 0.757812, sec/iter 0.015381
[tst] ep 3, step 1521, accu 0.796875, sec/iter 0.005513
[trn] ep 4, step 1750, loss 0.286988, accu 0.828125, sec/iter 0.013617
[tst] ep 4, step 2028, accu 0.820312, sec/iter 0.005127
[trn] ep 5, step 2250, loss 0.240008, accu 0.851562, sec/iter 0.015156
[tst] ep 5, step 2535, accu 0.835938, sec/iter 0.005836
[trn] ep 6, step 2750, loss 0.208887, accu 0.867188, sec/iter 0.013542
[tst] ep 6, step 3042, accu 0.859375, sec/iter 0.005498
[trn] ep 7, step 3250, loss 0.183400, accu 0.882812, sec/iter 0.015910
[tst] ep 7, step 3549, accu 0.851562, sec/iter 0.007850
[trn] ep 8, step 3750, loss 0.164586, accu 0.882812, sec/iter 0.014953
[tst] ep 8, step 4056, accu 0.867188, sec/i

In [20]:
# !tensorboard --ip 0.0.0.0 --logdir logdir

## Test #5 - LayerNormBasicLSTMCell

<div  style="border:1px solid black;border-radius:5px">
<code>
    cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                        num_hidden_units)
 
</code>
</div>

In [21]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 tf.contrib.rnn.LayerNormBasicLSTMCell)

In [22]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter('logdir/train_ln_basic_lstm',
                                     graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter('logdir/test_ln_basic_lstm',
                                     graph=tf.get_default_graph())

train(inputs_, labels_, 1, train_writer, test_writer)

[trn] ep 1, step 250, loss nan, accu 0.039062, sec/iter 0.063241
[tst] ep 1, step 507, accu 0.054688, sec/iter 0.011702


In [23]:
# !tensorboard --ip 0.0.0.0 --logdir logdir

## Test #6 - LayerNormBasicLSTMCell - what's wrong?

- [`tf.check_numerics()`](http://devdocs.io/tensorflow~python/tf/check_numerics)
> When run, reports an `InvalidArgument` error if tensor has any values that are not a number (NaN) or infinity (Inf). Otherwise, passes tensor as-is.

```
    check_numerics(
        tensor,
        message,
        name=None
    )
```


- [`tf.gradients()`](http://devdocs.io/tensorflow~python/tf/gradients)
> Constructs symbolic partial derivatives of sum of `ys` w.r.t. x in `xs`

```
    gradients(
        ys,
        xs,
        grad_ys=None,
        name='gradients',
        colocate_gradients_with_ops=False,
        gate_gradients=False,
        aggregation_method=None
    )
```


- 사용 예:
```
    # check_numerics
    self.check = [tf.check_numerics(t,
                    'check_numerics: {}'.format(t.name)) \
                  for t in tf.gradients(
                              loss,
                              tf.trainable_variables()) \
                  if t is not None]
    ...
    summary, optimize, loss, accuracy, _ = \
        sess.run([model.train_summary,
                  model.optimize,
                  model.loss,
                  model.accuracy,
                  model.check],
                  ...
```


In [24]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 tf.contrib.rnn.LayerNormBasicLSTMCell,
                 add_check = True)

# 여기서 NaN 문제가 생기는 걸 확인했으면, 다음 방법들을 시도 해 보세요
#  1. learning_rate 를 줄여
#       e.g.: lr = 0.0001
#  2. 적당한 값으로 gradient clipping
#       e.g.: use_grad_clip = True
#  3. 사용한 컴퍼넌트에 smoothing 할 수 있는 파라메터가 있는지 확인하고 적용
#     rnn_cell_class = lambda num_hidden_units: \
#        tf.contrib.rnn.LayerNormBasicLSTMCell(
#                             num_hidden_units,
#                             norm_gain=0.85,
#                             norm_shift=0.15)

In [ ]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter(
                'logdir/train_ln_basic_lstm_2',
                graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter(
                'logdir/test_ln_basic_lstm_2',
                graph=tf.get_default_graph())

train(inputs_, labels_, 10, train_writer, test_writer)

<div style="padding:5px;border:1px solid black;border-radius:5px">
```
InvalidArgumentError: check_numerics: gradients_1/rnn/while/rnn/layer_norm_basic_lstm_cell/state_1/batchnorm/sub/Enter_grad/b_acc_3:0 : Tensor had NaN values
	 [[Node: CheckNumerics_10 = CheckNumerics[T=DT_FLOAT, message="check_numerics: gradients_1/rnn/while/rnn/layer_norm_basic_lstm_cell/state_1/batchnorm/sub/Enter_grad/b_acc_3:0", _device="/job:localhost/replica:0/task:0/cpu:0"](gradients_1/rnn/while/rnn/layer_norm_basic_lstm_cell/state_1/batchnorm/sub/Enter_grad/b_acc_3)]]


```
</div>

In [26]:
# !tensorboard --ip 0.0.0.0 --logdir logdir

In [27]:
tf.reset_default_graph()

inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

model = MnistRnn(inputs_,
                 labels_,
                 INPUT_UNITS,
                 NUM_HIDDEN_UNITS,
                 BATCH_SIZE,
                 MAX_SEQ_LEN,
                 lambda num_hidden_units: \
                       tf.contrib.rnn.LayerNormBasicLSTMCell(
                                            num_hidden_units,
                                            norm_gain=0.85,
                                            norm_shift=0.15),
                 add_check = True,
                 lr = 0.0001,
                 use_grad_clip = True)

In [28]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

train_writer = tf.summary.FileWriter(
                'logdir/train_ln_basic_lstm_2',
                graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter(
                'logdir/test_ln_basic_lstm_2',
                graph=tf.get_default_graph())

train(inputs_, labels_, 10, train_writer, test_writer)

[trn] ep 1, step 250, loss 2.108183, accu 0.046875, sec/iter 0.104071
[tst] ep 1, step 507, accu 0.421875, sec/iter 0.012975
[trn] ep 2, step 750, loss 1.525936, accu 0.429688, sec/iter 0.104317
[tst] ep 2, step 1014, accu 0.515625, sec/iter 0.009750
[trn] ep 3, step 1250, loss 1.210164, accu 0.539062, sec/iter 0.103209
[tst] ep 3, step 1521, accu 0.601562, sec/iter 0.008972
[trn] ep 4, step 1750, loss 0.963128, accu 0.632812, sec/iter 0.088096
[tst] ep 4, step 2028, accu 0.671875, sec/iter 0.008804
[trn] ep 5, step 2250, loss 0.794114, accu 0.664062, sec/iter 0.087969
[tst] ep 5, step 2535, accu 0.687500, sec/iter 0.010850
[trn] ep 6, step 2750, loss 0.673972, accu 0.726562, sec/iter 0.091234
[tst] ep 6, step 3042, accu 0.726562, sec/iter 0.008794
[trn] ep 7, step 3250, loss 0.574158, accu 0.750000, sec/iter 0.092356
[tst] ep 7, step 3549, accu 0.742188, sec/iter 0.008645
[trn] ep 8, step 3750, loss 0.494440, accu 0.750000, sec/iter 0.088173
[tst] ep 8, step 4056, accu 0.757812, sec/i

## 정리해 봅시다

### 텐서플로우에서 지원하는 RNN Cell 유형

- [`tf.contrib.rnn.BasicRNNCell()`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/basicrnncell)

- [`tf.contrib.rnn.BasicLSTMCell()`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/basiclstmcell)

- [`tf.contrib.rnn.LSTMCell()`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/lstmcell)

- [`tf.contrib.rnn.GRUCell()`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/grucell)

- [`tf.contrib.rnn.LayerNormBasicLSTMCell()`](http://devdocs.io/tensorflow~python/tf/contrib/rnn/layernormbasiclstmcell)


### RNN 구성

- [`tf.nn.dynamic_rnn()`](http://devdocs.io/tensorflow~python/tf/nn/dynamic_rnn)
```
    cell         = tf.contrib.rnn.BasicRNNCell(
                    num_hidden_units)
    last, states = tf.nn.dynamic_rnn(
                        cell, 
                        inputs, 
                        sequence_length=sequence_length,
                        dtype=tf.float32)
```


### 실행중 NaN, Inf 등이 발생하면?

1. [`tf.check_numerics()`](http://devdocs.io/tensorflow~python/tf/check_numerics)로 문제가 되는 컴퍼넌트 파악
1. learning_rate 를 줄여본다
1. 적당한 값으로 gradient clipping 해 본다
1. 사용한 컴퍼넌트에 smoothing 할 수 있는 파라메터가 있는지 확인하고 적용해 본다

### gradient clipping?

- [`tf.clip_by_global_norm`](http://devdocs.io/tensorflow~python/tf/clip_by_global_norm)
```
    clip_by_global_norm(
        t_list,
        clip_norm,
        use_norm=None,
        name=None
    )
```

- [`tf.gradients`](http://devdocs.io/tensorflow~python/tf/gradients)
```
    gradients(
        ys,
        xs,
        grad_ys=None,
        name='gradients',
        colocate_gradients_with_ops=False,
        gate_gradients=False,
        aggregation_method=None
    )
```

- [`optimizer.apply_gradients`](http://devdocs.io/tensorflow~python/tf/train/optimizer#apply_gradients)
```
    apply_gradients(
        grads_and_vars,
        global_step=None,
        name=None
    )
```

- 사용예:
```
    tvars_     = tf.trainable_variables()
    grads_, _  = tf.clip_by_global_norm(
                    tf.gradients(loss,tvars_),
                    5.0)
    optimize   = tf.train.AdamOptimizer(
                    learning_rate=learning_rate) \
                    .apply_gradients(zip(grads_, tvars_))
```
